In [1]:
import os
import time

from joblib import dump, load

import matplotlib.pyplot as plt
from IPython import display

from imageio import imread
import numpy as np 

import theano.tensor as T
import lasagne.layers
import theano

from tqdm import tqdm_notebook

from nn_utils import build_UNet, iterate_minibatches
from train_data_preparation import prepare_data, extract_random_patches, extract_grid_patches

In [2]:
run processing_functions.ipynb

In [3]:
patch_size = 64
base_n_filters = 32
learning_rate = 1e-3
import lasagne.nonlinearities 
nonlinearity = lasagne.nonlinearities.elu
num_epochs = 10
batch_size = 128

In [4]:
weights_name = '/home/efim/Study/nn_seg/output_data/weights/Kamchatka_basalt+ceramics_weights_iou/UNet_params_ep003.jbl'

In [5]:
input_image = T.tensor4('input')
target = T.matrix('output', dtype='int64')

net = build_UNet(n_input_channels=1, BATCH_SIZE=None, num_output_classes=2,
                 pad='same', nonlinearity=lasagne.nonlinearities.elu,
                 input_dim=(patch_size, patch_size), base_n_filters=base_n_filters, do_dropout=True,
                 weights=load(weights_name))
predictions = lasagne.layers.get_output(net['output_flattened'], input_image)
get_predictions = theano.function([input_image], predictions)

In [6]:
input_stack_path = '/home/efim/Study/nn_seg/binary_data/Urna_22/NLM'
output_stack_path = '/home/efim/Study/nn_seg/binary_data/Urna_22/CAC'

In [ ]:
%%time
input_stack = []
output_stack = []
results = []
for i, fname in enumerate(sorted(os.listdir(input_stack_path))):
    input_img = imread(os.path.join(input_stack_path, fname))
    input_stack.append(input_img.reshape(input_img.shape + (1, )))
    
    output_img = imread(os.path.join(output_stack_path, fname))
    output_stack.append(output_img.reshape(output_img.shape + (1, )))
    
    prediction = predict_image(input_img, get_predictions, patch_size, n_input_channels=1)
    prediction = prediction.reshape(prediction.shape + (1, ))
    results.append(prediction)
    if i > 2:
        break
        
input_stack = np.concatenate(input_stack, axis=2)
output_stack = np.concatenate(output_stack, axis=2)
prediction = np.concatenate(results, axis=2)

In [8]:
plt.imshow(prediction, cmap='gray')

NameError: name 'prediction' is not defined